<a href="https://colab.research.google.com/github/gautamHCSCV/Machine-Learning-Labs/blob/main/Perceptual_vs_FBL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 75 kB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Virtual env/a_data_perceptual_fbl/perceptual/dragonskin/CatBoost/'
X_train = pd.read_csv(path+'pos_percep_cb_train_dragon.csv')
X_test = pd.read_csv(path+'pos_percep_cb_test_dragon.csv')
Y_train = pd.read_csv(path+'force_percep_cb_train_dragon.csv')
Y_test = pd.read_csv(path+'force_percep_cb_test_dragon.csv')
X_train.shape,Y_train.shape

((1243, 3), (1243, 1))

In [ ]:
from scipy import *
from scipy.linalg import norm, pinv
import math


class RBF:

        def __init__(self, indim, numCenters, outdim):
            self.indim = indim
            self.outdim = outdim
            self.numCenters = numCenters
            self.centers = [np.random.uniform(-1, 1, indim) for i in range(numCenters)]
            self.beta = 8
            self.W = np.random.random((self.numCenters, self.outdim))

        def _basisfunc(self, c, d):
            assert len(d) == self.indim
            return norm(c-d)**3

        def _calcAct(self, X):
            # calculate activations of RBFs
            G = np.zeros((X.shape[0], self.numCenters), float)
            for ci, c in enumerate(self.centers):
                for xi, x in enumerate(X):
                    G[xi,ci] = self._basisfunc(c, x)
            return G

        def train(self, X, Y):
            """ X: matrix of dimensions n x indim 
                y: column vector of dimension n x 1 """

            # choose random center vectors from training set
            rnd_idx = np.random.permutation(X.shape[0])[:self.numCenters]
            self.centers = [X[i,:] for i in rnd_idx]

            #print("center", self.centers)
            # calculate activations of RBFs
            G = self._calcAct(X)
            #print(G)

            # calculate output weights (pseudoinverse)
            self.W = np.dot(pinv(G), Y)

        def test(self, X):
            """ X: matrix of dimensions n x indim """

            G = self._calcAct(X)
            Y = np.dot(G, self.W)
            return Y

# Perceptual

In [ ]:
x_train,y_train = np.array(X_train),np.array(Y_train)
x_test,y_test = np.array(X_test), np.array(Y_test)
x_test.shape

(159833, 3)

In [ ]:
t1 = datetime.now()
rbf = RBF(3,50, 1)
rbf.train(x_train, y_train)
t2 = datetime.now()

y1 = rbf.test(x_test)
print('time taken by RBF-perceptual:', t2-t1)
print('Root Mean Squared error: ', mean_squared_error(y_test,y1)**0.5)
    
statement1 = 'time taken by RBF-perceptual: {} \nRoot Mean Squared error: {}\n'.format(t2-t1, mean_squared_error(y_test,y1)**0.5)

time taken by RBF-perceptual: 0:00:00.646386
Root Mean Squared error:  0.1270594511340718


In [ ]:
cat = CatBoostRegressor(iterations=1200, learning_rate=0.01, random_seed=42)
if 1==1:
    t1 = datetime.now()
    cat.fit(x_train,y_train)
    t2 = datetime.now()

    y2 = cat.predict(x_test)
    print('time taken by Catboost on perceptual data:', t2-t1)
    print('Root Mean Squared error: ', mean_squared_error(y_test,y2)**0.5)
    
    statement2 = 'time taken by catboost: {} \nRoot Mean Squared error: {}\n'.format(t2-t1, mean_squared_error(y_test,y2)**0.5)
    

Learning rate set to 0.042375
0:	learn: 0.9849584	total: 4.16ms	remaining: 4.16s
1:	learn: 0.9496312	total: 8.03ms	remaining: 4.01s
2:	learn: 0.9150509	total: 10.9ms	remaining: 3.61s
3:	learn: 0.8816357	total: 15.8ms	remaining: 3.94s
4:	learn: 0.8502004	total: 18.5ms	remaining: 3.69s
5:	learn: 0.8193089	total: 23ms	remaining: 3.82s
6:	learn: 0.7897525	total: 25.1ms	remaining: 3.56s
7:	learn: 0.7616837	total: 28.5ms	remaining: 3.54s
8:	learn: 0.7362119	total: 32.1ms	remaining: 3.53s
9:	learn: 0.7098002	total: 35.7ms	remaining: 3.53s
10:	learn: 0.6841714	total: 39.8ms	remaining: 3.58s
11:	learn: 0.6600821	total: 41.5ms	remaining: 3.42s
12:	learn: 0.6364867	total: 44.4ms	remaining: 3.37s
13:	learn: 0.6138472	total: 45.8ms	remaining: 3.23s
14:	learn: 0.5934697	total: 48.8ms	remaining: 3.2s
15:	learn: 0.5738018	total: 50.7ms	remaining: 3.12s
16:	learn: 0.5539005	total: 54.2ms	remaining: 3.13s
17:	learn: 0.5341330	total: 70.7ms	remaining: 3.86s
18:	learn: 0.5149460	total: 73ms	remaining: 3.7

In [ ]:
path = '/content/drive/MyDrive/Virtual env/a_data_perceptual_fbl/perceptual/dragonskin/RF/'
X_train = pd.read_csv(path+'pos_percep_rf_train_dragon.csv')
X_test = pd.read_csv(path+'pos_percep_rf_test_dragon.csv')
Y_train = pd.read_csv(path+'force_percep_rf_train_dragon.csv')
Y_test = pd.read_csv(path+'force_percep_rf_test_dragon.csv')
x_train,y_train = np.array(X_train),np.array(Y_train)
x_test,y_test = np.array(X_test), np.array(Y_test)
x_test.shape, x_train.shape

((159833, 11), (1243, 11))

In [ ]:
rf = RandomForestRegressor()
if 1==1:
    t1 = datetime.now()
    rf.fit(x_train,y_train)
    t2 = datetime.now()

    y3 = rf.predict(x_test)
    print('time taken by Random Forest on perceptual data:', t2-t1)
    print('Root Mean Squared error: ', mean_squared_error(y_test,y3)**0.5)
    
    statement3 = 'time taken by Random Forest: {} \nRoot Mean Squared error: {}\n'.format(t2-t1, mean_squared_error(y_test,y3)**0.5)

time taken by Random Forest on perceptual data: 0:00:01.466696
Root Mean Squared error:  0.16658116160609204


# FBL

In [ ]:
path = '/content/drive/MyDrive/Virtual env/a_data_perceptual_fbl/fbl/dragonskin/CatBoost/'
X_train = pd.read_csv(path+'pos_fbl_cb_train_dragon.csv')
X_test = pd.read_csv(path+'pos_fbl_cb_test_dragon.csv')
Y_train = pd.read_csv(path+'force_fbl_cb_train_dragon.csv')
Y_test = pd.read_csv(path+'force_fbl_cb_test_dragon.csv')
X_train.shape,Y_train.shape

((23242, 3), (23242, 1))

In [ ]:
x_train,y_train = np.array(X_train),np.array(Y_train)
x_test,y_test = np.array(X_test), np.array(Y_test)
x_test.shape

(159833, 3)

In [ ]:
t1 = datetime.now()
rbf = RBF(3,50, 1)
rbf.train(x_train, y_train)
t2 = datetime.now()

y1 = rbf.test(x_test)
print('time taken by RBF-FBL:', t2-t1)
print('Root Mean Squared error: ', mean_squared_error(y_test,y1)**0.5)
    
statement1 = 'time taken by RBF-FBL: {} \nRoot Mean Squared error: {}\n'.format(t2-t1, mean_squared_error(y_test,y1)**0.5)

time taken by RBF-FBL: 0:00:09.005536
Root Mean Squared error:  0.2305602414215361


In [ ]:
cat = CatBoostRegressor(iterations=1200, learning_rate=0.01, random_seed=42)
if 1==1:
    t1 = datetime.now()
    cat.fit(x_train,y_train)
    t2 = datetime.now()

    y2 = cat.predict(x_test)
    print('time taken by Catboost on FBL data:', t2-t1)
    print('Root Mean Squared error: ', mean_squared_error(y_test,y2)**0.5)
    
    statement2 = 'time taken by catboost: {} \nRoot Mean Squared error: {}\n'.format(t2-t1, mean_squared_error(y_test,y2)**0.5)
    

0:	learn: 1.3708555	total: 57.6ms	remaining: 1m 9s
1:	learn: 1.3574326	total: 70ms	remaining: 41.9s
2:	learn: 1.3442845	total: 81.2ms	remaining: 32.4s
3:	learn: 1.3311860	total: 92.4ms	remaining: 27.6s
4:	learn: 1.3183524	total: 103ms	remaining: 24.6s
5:	learn: 1.3054917	total: 114ms	remaining: 22.6s
6:	learn: 1.2929444	total: 124ms	remaining: 21.1s
7:	learn: 1.2804950	total: 135ms	remaining: 20.1s
8:	learn: 1.2682033	total: 146ms	remaining: 19.3s
9:	learn: 1.2558984	total: 156ms	remaining: 18.6s
10:	learn: 1.2437013	total: 168ms	remaining: 18.1s
11:	learn: 1.2315397	total: 179ms	remaining: 17.7s
12:	learn: 1.2195703	total: 190ms	remaining: 17.3s
13:	learn: 1.2078721	total: 201ms	remaining: 17s
14:	learn: 1.1962286	total: 211ms	remaining: 16.7s
15:	learn: 1.1845469	total: 218ms	remaining: 16.1s
16:	learn: 1.1729963	total: 225ms	remaining: 15.7s
17:	learn: 1.1615997	total: 237ms	remaining: 15.6s
18:	learn: 1.1504637	total: 246ms	remaining: 15.3s
19:	learn: 1.1391796	total: 260ms	remaini

In [ ]:
path = '/content/drive/MyDrive/Virtual env/a_data_perceptual_fbl/fbl/dragonskin/RF/'
X_train = pd.read_csv(path+'pos_fbl_rf_train_dragon.csv')
X_test = pd.read_csv(path+'pos_fbl_rf_test_dragon.csv')
Y_train = pd.read_csv(path+'force_fbl_rf_train_dragon.csv')
Y_test = pd.read_csv(path+'force_fbl_rf_test_dragon.csv')
x_train,y_train = np.array(X_train),np.array(Y_train)
x_test,y_test = np.array(X_test), np.array(Y_test)
x_test.shape, x_train.shape

((159833, 11), (23242, 11))

In [ ]:
rf = RandomForestRegressor()
if 1==1:
    t1 = datetime.now()
    rf.fit(x_train,y_train)
    t2 = datetime.now()

    y3 = rf.predict(x_test)
    print('time taken by Random Forest on perceptual data:', t2-t1)
    print('Root Mean Squared error: ', mean_squared_error(y_test,y3)**0.5)
    
    statement3 = 'time taken by Random Forest: {} \nRoot Mean Squared error: {}\n'.format(t2-t1, mean_squared_error(y_test,y3)**0.5)

time taken by Random Forest on perceptual data: 0:00:21.004786
Root Mean Squared error:  0.15271268650547493
